In [ ]:
# Goal => make one file/sector concatenating each file in a sector
#      => modify the shape adding 3 colonnes : Year, compound(gas) and Process Group(sector)
import pandas as pd

import glob


# tree structure source folder ./EDGAR_Import/
#                                             EDGAR_excl_short_cycle/
#                                                                    AGS_txt/
#                                                                    CHE_txt/
#                                                                    ...
#                                             EDGAR_short_cycle/
#                                                               AWB_txt/
#                                                               ENE_txt/
#                                                             ...
# tree structure destination folder ./EDGAR_merge/

folder_list = glob.glob("./EDGAR_Import/*/*/")

for f in folder_list:
    
    #get the sector from folder name
    sector = f.replace('_txt','').split("/")[3]
    #print(sector)
    #get file list from folder
    file_list = glob.glob(f+"v50_CO2*.txt")
    
    for i in range(len(file_list)):
        
        #get exclude short cycle or not
        list_type = [m.split("/")[4].split("_")[2] for m in file_list]
        
        #skip the two first lines which are just informative
        df_eachdata = pd.read_csv(file_list[i],sep=";", skiprows=[0,1])
        
        #delete the year in column name to make unique column name
        df_eachdata.rename(columns={ df_eachdata.columns[2]: "emission (tons)" }, inplace = True)
        
        #get second line to make columns from part of it
        df_eachinfo = pd.read_csv(file_list[i],sep=";", skiprows=[0,2], nrows=0)
    
        #delete space from column name
        df_eachinfo.columns = df_eachinfo.columns.str.strip()
        
        #split column name and get each part in a an b variable
        a = df_eachinfo.columns.str.split(":").str[0]
        b = df_eachinfo.columns.str.split(":").str[1]
        
        #make df with a variable as key and b as value
        df_eachinfo = pd.DataFrame(dict(zip(a,b)), index=[0])
        
        #keep year column and Compound column (gas). As process group info is not in all file, the column is create
        #after from the current folder name.
        df_eachinfo = df_eachinfo[["Year","Compound"]]
        
        #cross join to add year and compound columns to the original data
        df_eachdata = df_eachinfo.assign(foo=1).merge(df_eachdata.assign(foo=1)).drop('foo', 1)
    
        if (i == 0):
            df_data = df_eachdata
    
        else:
            df_data = pd.concat([df_data, df_eachdata])
       
    #Add column "Process Group" from folder name 
    df_data['Process Group'] = sector
    
    
    emission_type = list_type[0]
    #print(file_list)
    
    #print(sector,df_eachinfo.columns)
    df_data.to_csv("./EDGAR_merge/v50_CO2_"+emission_type+"_short-cycle_"+sector+"_all_year.csv")

